<a href="https://colab.research.google.com/github/ibadrather/pytorch_learn/blob/main/Part%2013%20-%20Multivariate_Time_Series_Data_Preprocessing_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet pytorch-lightning
!pip install --quiet tqdm

     |████████████████████████████████| 585 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
     |████████████████████████████████| 140 kB 57.4 MB/s 
     |████████████████████████████████| 419 kB 46.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.7 MB/s 
     |████████████████████████████████| 271 kB 37.3 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 144 kB 26.7 MB/s 


In [3]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

### Styling Settings

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10

tqdm.pandas()


In [5]:
# Random Seed Pytorch Lightning
pl.seed_everything(42)

Global seed set to 42


42

## Load Data

In [8]:
# Mouting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Dataset Source: https://www.cryptodatadownload.com/data/binance/

In [32]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Binance_BTCUSDT_minute.csv"
df = pd.read_csv(data_path,)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,,,,,,,,
1655511360000,2022-06-18 00:16:00,BTC/USDT,20597.03000000,20599.29000000,20568.90000000,20599.28000000,64.92620000,1336133.91370160
1655511300000,2022-06-18 00:15:00,BTC/USDT,20619.34000000,20645.13000000,20597.02000000,20597.02000000,56.02534000,1155457.13070930
1655511240000,2022-06-18 00:14:00,BTC/USDT,20619.22000000,20638.02000000,20613.27000000,20619.33000000,67.57083000,1393748.88414450
1655511180000,2022-06-18 00:13:00,BTC/USDT,20643.04000000,20643.05000000,20617.26000000,20619.22000000,58.09482000,1198367.01320070
1655511120000,2022-06-18 00:12:00,BTC/USDT,20593.62000000,20665.66000000,20590.02000000,20643.05000000,215.50463000,4447087.30072560
...,...,...,...,...,...,...,...,...
1567965660000,2019-09-08 18:01:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.0,0.0
1567965600000,2019-09-08 18:00:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.0,0.0
1567965540000,2019-09-08 17:59:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.001,10.0
1567965480000,2019-09-08 17:58:00,BTC/USDT,10000.0,10000.0,10000.0,10000.0,0.0,0.0


In [21]:
df = df.sort_values(by=["date"]).reset_index()
df.head()

KeyError: ignored

In [24]:
df.columns

Index([], dtype='object')

# New Section